In [1]:
# BEFORE RUNNING, INSTALL THE PACKAGES BELOW
import pandas as pd
import folium
import geopandas
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

In [6]:
! pip install geopy

     |████████████████████████████████| 113 kB 543 kB/s eta 0:00:01
  Using cached geographiclib-1.50-py3-none-any.whl (38 kB)


In [4]:
! pip install geopandas

  Using cached geopandas-0.7.0-py2.py3-none-any.whl (928 kB)
     |████████████████████████████████| 10.9 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 14.7 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 2.9 MB/s eta 0:00:01
  Using cached cligj-0.5.0-py3-none-any.whl (5.7 kB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)


In [2]:
! pip install folium

     |████████████████████████████████| 93 kB 401 kB/s eta 0:00:011


In [2]:
# Import data.  Note no deduplication is being done
miami_sales = pd.read_csv('miami_sales_chain_jurisdictions_051520.csv', low_memory=False)
miami_chained_offshore = pd.read_csv('miami_sales_chain_jurisdictions_offshore_051520.csv')
miami_POI_final = pd.read_csv('/home/cecil/data/sunlight/miami_POI_mapping_final.csv')
miami_ICIJ_matches = pd.read_csv('miami_sales_icij_matches_all_052720.csv')

In [17]:
miami_sales.head()

,Unnamed: 0,Unnamed: 0.1,Folio,Municipality,SaleID,OR_BK,OR_PG,TransferCode,Document Type,Grantor(seller),Grantee(buyer),DateOfSale,Price,VI,QU_FLG,DORCode,SiteAddress,StreetNumber,StreetPrefix,StreetName,StreetNumberSuffix,StreetSuffix,StreetDirection,CondoUnit,SiteCity,SiteZip,MailingAddressLine1,MailingAddressLine2,MailingAddressLine3,City,State,Zip,Country,SalesCode,Owner1,Owner2,Owner3,Zoning,AdjustedSqFt,LotSF,Acres,Bedrooms,Baths,1/2 Baths,LivingUnits,Stories,NumberOfBuilding,YearBuilt,EffectiveYearBuilt,LivingSqFt,ActualSqFt,BUYER_COR_NAME,BUYER_Jurisdictions,BUYER_Joined_name,BUYER_Similarity_Score,SELLER_COR_NAME,SELLER_Jurisdictions,SELLER_Joined_name,SELLER_Similarity_Score,POI_buyer,POI_seller,POI_own1,POI_mailaddy,POI_siteaddy,POI_ALL,address_combined
0,0,0,1,132310260010,1,5,30495,3146,WDE,PLACIDO DIAZ III,CARLOS M GARCIA,04-10-17 0:00,3600000,I,Q,101,831 N VENETIAN DR,831.0,N,VENETIAN,NaN,DR,NaN,NaN,Miami,33139-1012,CARLOS M GARCIA,NaN,PO BOX 402566,MIAMI,FL,33140,USA,1,CARLOS M GARCIA,NaN,NaN,100,3077,11250.0,0.2583,5,2.0,0,1,1,1,1949,1989,3077,3077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,831 N VENETIAN DR 33139-1012 Miami USA
1,1,1,2,132310260010,1,4,27545,978,QCD,PLACIDO DIAZ III,PLACIDO DIAZ III,12/22/2010 12:00:00 AM,100,I,U,101,831 N VENETIAN DR,831.0,N,VENETIAN,NaN,DR,NaN,NaN,Miami,33139-1012,CARLOS M GARCIA,NaN,PO BOX 402566,MIAMI,FL,33140,USA,11,CARLOS M GARCIA,NaN,NaN,100,3077,11250.0,0.2583,5,2.0,0,1,1,1,1949,1989,3077,3077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,831 N VENETIAN DR 33139-1012 Miami USA
2,2,2,3,132310260010,1,1,18157,877,NaN,NaN,NaN,06-01-98 0:00,325000,I,Q,101,831 N VENETIAN DR,831.0,N,VENETIAN,NaN,DR,NaN,NaN,Miami,33139-1012,CARLOS M GARCIA,NaN,PO BOX 402566,MIAMI,FL,33140,USA,0,CARLOS M GARCIA,NaN,NaN,100,3077,11250.0,0.2583,5,2.0,0,1,1,1,1949,1989,3077,3077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,831 N VENETIAN DR 33139-1012 Miami USA
3,3,3,4,132310260010,1,2,18105,1711,NaN,NaN,NaN,04-01-98 0:00,350000,I,Q,101,831 N VENETIAN DR,831.0,N,VENETIAN,NaN,DR,NaN,NaN,Miami,33139-1012,CARLOS M GARCIA,NaN,PO BOX 402566,MIAMI,FL,33140,USA,0,CARLOS M GARCIA,NaN,NaN,100,3077,11250.0,0.2583,5,2.0,0,1,1,1,1949,1989,3077,3077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,831 N VENETIAN DR 33139-1012 Miami USA
4,4,4,5,132310260010,1,3,18105,1709,NaN,NaN,NaN,04-01-98 0:00,0,,U,101,831 N VENETIAN DR,831.0,N,VENETIAN,NaN,DR,NaN,NaN,Miami,33139-1012,CARLOS M GARCIA,NaN,PO BOX 402566,MIAMI,FL,33140,USA,1,CARLOS M GARCIA,NaN,NaN,100,3077,11250.0,0.2583,5,2.0,0,1,1,1,1949,1989,3077,3077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,831 N VENETIAN DR 33139-1012 Miami USA


In [6]:
# Creates a new 'address_combined' column which is (should be!) geocode-friendly
miami_sales['address_combined'] = miami_sales.apply (lambda row: str(row.StreetNumber).replace('nan', '').split('.')[0] + ' ' + str(row.StreetPrefix).replace('nan', '') + ' ' + str(row.StreetName).replace('nan', '') + ' ' + str(row.StreetSuffix).replace('nan', '') + ' ' + str(row.SiteZip).replace('nan', '') + ' ' + str(row.SiteCity).replace('nan', '') + ' ' + 'USA', axis=1)

In [104]:
# Isolate 'exposed' offshore ownership
chain_laundering = ['BZ', 'VG', 'DE', 'RU', 'SE']
country_laundering = ['AZERBAIJAN', 'BAHAMAS','BELIZE','BRITISH VIRGIN ISLANDS','CAYMAN ISLANDS','CYPRUS','GUERNSEY','ISLE OF MAN','NETHERLANDS ANTILLES','PANAMA','RUSSIA','SAINT KITTS AND NEVIS','SINT MAARTEN','TRINIDAD AND TOBAGO','VIRGIN ISLANDS']

miami_sales_offshore =  miami_sales.loc[miami_sales['Country'].isin(country_laundering)]

In [105]:
# Create a smaller subset of the transactions to geocode and map.  Modify to select other interesting transactions!
# In this case, isolate 'oligarchy; transactions from the larger initial POI set
miami_sales_oligarachy = miami_sales.dropna(subset=['POI_ALL'])

In [5]:
# Initialize & test-drive geocoder
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("831 N VENETIAN DR Miami Florida 33139-1012, USA")
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

831, North Venetian Drive, Buena Vista, Miami, Miami-Dade County, Florida, 33139, United States of America
Latitude = 25.79076495, Longitude = -80.17852054345659


In [10]:
# Geocode addresses.  For a larger dataset, this will take a while, so saving results is recommended
# Clone for each data-stream b/c I'm too lazy to write a func def
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)
miami_sales_oligarachy['location'] = miami_sales_oligarachy['address_combined'].apply(geocode)
miami_sales_oligarachy['point'] = miami_sales_oligarachy['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [11]:
miami_sales_offshore['location'] = miami_sales_offshore['address_combined'].apply(geocode)
miami_sales_offshore['point'] = miami_sales_offshore['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [91]:
miami_chained_offshore['address_combined'] = miami_chained_offshore.apply (lambda row: str(row.StreetNumber).replace('nan', '').split('.')[0] + ' ' + str(row.StreetPrefix).replace('nan', '') + ' ' + str(row.StreetName).replace('nan', '') + ' ' + str(row.StreetSuffix).replace('nan', '') + ' ' + str(row.SiteZip).replace('nan', '') + ' ' + str(row.SiteCity).replace('nan', '') + ' ' + 'USA', axis=1)
miami_chained_offshore['location'] = miami_chained_offshore['address_combined'].apply(geocode)
miami_chained_offshore['point'] = miami_chained_offshore['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [76]:
miami_ICIJ_matches['address_combined'] = miami_ICIJ_matches.apply (lambda row: str(row.StreetNumber).replace('nan', '').split('.')[0] + ' ' + str(row.StreetPrefix).replace('nan', '') + ' ' + str(row.StreetName).replace('nan', '') + ' ' + str(row.StreetSuffix).replace('nan', '') + ' ' + str(row.SiteZip).replace('nan', '') + ' ' + str(row.SiteCity).replace('nan', '') + ' ' + 'USA', axis=1)
miami_ICIJ_matches['location'] = miami_ICIJ_matches['address_combined'].apply(geocode)
miami_ICIJ_matches['point'] = miami_ICIJ_matches['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [65]:
# Note that miami_POI_final is a handcrafted stub dataset, representing only those priority investigative targets mentioned in the publication
miami_POI_final['address_combined'] = miami_POI_final.apply (lambda row: str(row.address) + ' Miami, Florida', axis=1)
miami_POI_final['location'] = miami_POI_final['address_combined'].apply(geocode)
miami_POI_final['point'] = miami_POI_final['location'].apply(lambda loc: tuple(loc.point) if loc else None)
miami_POI_final.head()

,name,address,value,color,address_combined,location,point
0,Yuriy Zhukov and Elena Sibgatullina,17749 Collins Ave 3501,7589500,blue,"17749 Collins Ave 3501 Miami, Florida","(The Mansions at Acqualina, Collins Avenue, Su...","(25.940047, -80.12050570145206, 0.0)"
1,Argo Family LLC,15901 Collins Ave 2606,1510000,red,"15901 Collins Ave 2606 Miami, Florida","(15901, Collins Avenue, Sunny Isles Beach, Mia...","(25.9236928, -80.121773, 0.0)"
2,Argo Family LLC,616 Ocean Blvd,1600000,red,"616 Ocean Blvd Miami, Florida","(616, Ocean Boulevard, Golden Beach, Miami-Dad...","(25.9728745, -80.11958304838168, 0.0)"
3,Argo Family 2704 LLC,16699 Collins Ave 2704,690000,red,"16699 Collins Ave 2704 Miami, Florida","(La Perla Ocean Residences, Collins Avenue, Su...","(25.930262550000002, -80.12136855, 0.0)"
4,Argo Family 2704 LLC,16699 Collins Ave 3203,810000,red,"16699 Collins Ave 3203 Miami, Florida","(La Perla Ocean Residences, Collins Avenue, Su...","(25.930262550000002, -80.12136855, 0.0)"


In [13]:
# Drop any uncoded rows (should be few to none) and pull geocoded lat/long into new columns.  still too lazy to write a def.
miami_sales_oligarachy_dropna = miami_sales_oligarachy.dropna(subset=['point'])
miami_sales_oligarachy_dropna['latitude'] = miami_sales_oligarachy_dropna.apply (lambda row: row.point[0], axis=1)
miami_sales_oligarachy_dropna['longitude'] = miami_sales_oligarachy_dropna.apply (lambda row: row.point[1], axis=1)
miami_sales_oligarachy_clean = miami_sales_oligarachy_dropna.drop_duplicates(subset='SiteAddress', keep="first")

In [69]:
miami_sales_offshore_dropna = miami_sales_offshore.dropna(subset=['point'])
miami_sales_offshore_dropna['latitude'] = miami_sales_offshore_dropna.apply (lambda row: row.point[0], axis=1)
miami_sales_offshore_dropna['longitude'] = miami_sales_offshore_dropna.apply (lambda row: row.point[1], axis=1)

In [80]:
miami_ICIJ_matches_dropna = miami_ICIJ_matches.dropna(subset=['point'])
miami_ICIJ_matches_dropna['latitude'] = miami_ICIJ_matches_dropna.apply (lambda row: row.point[0], axis=1)
miami_ICIJ_matches_dropna['longitude'] = miami_ICIJ_matches_dropna.apply (lambda row: row.point[1], axis=1)

In [92]:
miami_chained_offshore_dropna = miami_chained_offshore.dropna(subset=['point'])
miami_chained_offshore_dropna['latitude'] = miami_chained_offshore_dropna.apply (lambda row: row.point[0], axis=1)
miami_chained_offshore_dropna['longitude'] = miami_chained_offshore_dropna.apply (lambda row: row.point[1], axis=1)

In [66]:
miami_POI_final = miami_POI_final.dropna(subset=['point'])
miami_POI_final['latitude'] = miami_POI_final.apply (lambda row: row.point[0], axis=1)
miami_POI_final['longitude'] = miami_POI_final.apply (lambda row: row.point[1], axis=1)

## MAPS!

In [102]:
# Plot geocoded lat/long points with the 'folium' module. 
all_offshore_properties = folium.Map(
    location=[25.8717, -80.1918],
#     tiles='cartodbpositron',
    zoom_start=11.5,
    )

miami_ICIJ_matches_dropna.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], tooltip=str(row['MailingAddressLine1'] + ' ::: ' + str(row['SiteAddress']) + ' $' + str(row['Price']))).add_to(all_offshore_properties), axis=1)
miami_sales_offshore_dropna.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], color = 'green', tooltip=str(row['MailingAddressLine1'] + ' ::: ' + str(row['SiteAddress']) + ' $' + str(row['Price']))).add_to(all_offshore_properties), axis=1)
miami_chained_offshore_dropna.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], color = 'red', tooltip=str(row['MailingAddressLine1'] + ' ::: ' + str(row['SiteAddress']) + ' $' + str(row['Price']))).add_to(all_offshore_properties), axis=1)
all_offshore_properties 

In [72]:
# Plot geocoded lat/long points with the 'folium' module.  
yeyo_stashes = folium.Map(
    location=[25.8717, -80.1918],
#     tiles='cartodbpositron',
    zoom_start=11.5,
    )

miami_POI_final.apply(lambda row:folium.Marker(
    location=[row["latitude"], row["longitude"]], 
    icon=folium.Icon(color=str(row['color']),icon="flag", prefix='fa'), 
    tooltip=str(row['name'] + ' ::: ' + str(row['address']) + ' $' + str(row['value']))).add_to(yeyo_stashes), axis=1)
yeyo_stashes

In [103]:
all_offshore_properties.save("Miami_offshore_all_sources_prototype_052920.html")